# Problema de asignación de trabajos de teoría.
### Autor: Saúl Sosa Díaz

En este cuaderno Jupyter, resolveremos un problema de asignación.

  



datos relevantes:
* Conjunto de grupos que llamaremos $G = \{1,\ldots,n\}$.
* Conjunto de trabajos que llamaremos $T = \{1,\ldots,m\}$.
* Preferencia que tiene un grupo específico por realizar otro trabajo específico. Lo denotaremos como $P_{gt}$. Donde cada posición indicará el la preferencia del grupo $g$ a realizar el trabajo $t$. $\forall g ∈ G \land \forall t ∈ T$

### Modelo
#### Variables.
* $X_{gt} = \begin{dcases}
   1 &\text{Si al grupo } g \text{ se le ha asignado el trabajo } t \text{ }\forall g ∈ G \land \forall t ∈ T\\
   0 &\text{Otro caso}
\end{dcases}$
#### Función Objetivo.
$$
\begin{array}{ccc}
min \sum_{g∈G}\sum_{t∈T} P_{gt} * X_{gt}& \\&  
\end{array}
$$
#### Restricciones.
$$
\begin{array}{ccc}
&  \sum_{t∈T} X_{gt} = 1 & \forall g \in G & \text{  Cada grupo tiene asignado un trabajo.} \\
&  \sum_{g∈G} X_{gt} <= 1 & \forall c \in C & \text{  Un trabajo puede estar asignado como mucho a un grupo.}\\
\end{array}
$$


## Celda para que Colab pueda ejecutar Julia

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

### Resolvemos el modelo.

#### Importamos las librerías necesarias.

In [59]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

using JuMP, GLPK, Random


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


#### Introducimos los datos.

Si se quisiera introducir datos de verdad bastaría con cambiar los datos de la siguiente celda.
Recordar que $P_{ij}$ indica la prioridad del grupo i por hacer el trabajo j. 

In [67]:
G = 11         # número de Grupos
T = 17        # número de Trabajos

# Define el valor para los trabajos no elegidos (número muy grande)
M = 100
Random.seed!(777)

P = zeros(G, T)
# Genera preferencias aleatorias sin repetir números en cada fila
for i in 1:G
    P[i, :] = randperm(T)
end

for i in 1:G
    for j in 1:T
        if P[i, j] > 5
            P[i, j] = M
        end
    end
end

P = Int.(P)

11×17 Matrix{Int64}:
 100  100  100  100  100    4  100  100  …  100  100  100  100    2    5  100
 100    5  100  100    2    4  100  100     100  100    1  100  100  100  100
 100  100  100    3    2    4  100    5     100  100  100  100    1  100  100
 100  100    4    1    3  100  100    2     100  100    5  100  100  100  100
 100  100  100  100    3    1  100  100     100    5    4  100  100    2  100
 100  100  100  100    3    1    2    4  …  100  100  100  100  100  100    5
 100    5  100  100  100  100  100    1       2    3  100  100  100  100  100
 100  100  100    1  100    5  100  100       2    3    4  100  100  100  100
   4    1  100  100  100  100    5  100     100  100  100  100    2    3  100
 100  100    2    1  100  100  100  100     100    4  100  100  100    3    5
 100  100    4  100    2  100    1  100  …  100    3    5  100  100  100  100

#### Definimos el modelo.

In [64]:
model = Model(GLPK.Optimizer)
set_silent(model)

# Definir variables
@variable(model, x[1:G,1:T] >= 0, binary=true)
# Definir función objetivo
@objective(model, Min, sum(P[g,t] * x[g,t] for g in 1:G, t in 1:T))

@constraint(model, c[g=1:G] , sum(x[g,:]) == 1 ) # Cada grupo tiene que tener asignado un trabjo.
@constraint(model, p[t=1:T] , sum(x[:,t]) <= 1 ) # Cada trabajo tiene que estar asignado como mucho a un grupo.


13-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 p[1] : x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] + x[6,1] + x[7,1] + x[8,1] + x[9,1] + x[10,1] + x[11,1] ≤ 1
 p[2] : x[1,2] + x[2,2] + x[3,2] + x[4,2] + x[5,2] + x[6,2] + x[7,2] + x[8,2] + x[9,2] + x[10,2] + x[11,2] ≤ 1
 p[3] : x[1,3] + x[2,3] + x[3,3] + x[4,3] + x[5,3] + x[6,3] + x[7,3] + x[8,3] + x[9,3] + x[10,3] + x[11,3] ≤ 1
 p[4] : x[1,4] + x[2,4] + x[3,4] + x[4,4] + x[5,4] + x[6,4] + x[7,4] + x[8,4] + x[9,4] + x[10,4] + x[11,4] ≤ 1
 p[5] : x[1,5] + x[2,5] + x[3,5] + x[4,5] + x[5,5] + x[6,5] + x[7,5] + x[8,5] + x[9,5] + x[10,5] + x[11,5] ≤ 1
 p[6] : x[1,6] + x[2,6] + x[3,6] + x[4,6] + x[5,6] + x[6,6] + x[7,6] + x[8,6] + x[9,6] + x[10,6] + x[11,6] ≤ 1
 p[7] : x[1,7] + x[2,7] + x[3,7] + x[4,7] + x[5,7] + x[6,7] + x[7,7] + x[8,7] + x[9,7] + x[10,7] + x[11,7] ≤ 1
 p[8] : x[1,8] + x[2,8] + x[3,8] + x[4,8] + x[5,8] 

#### Resolvemos el modelo.

In [65]:
optimize!(model)

#### Mostramos el resultado.

In [66]:
objetive = objective_value(model)

for g in 1:G  
    for t in 1:T
            if value(x[g,t]) > 0
                println("El grupo $g se le asigna el trabajo $t, inicialmente la prioridad era ", P[g,t])
            end
    end
    println()
end

El grupo 1 se le asigna el trabajo 10, inicialmente la prioridad era 1

El grupo 2 se le asigna el trabajo 4, inicialmente la prioridad era 1

El grupo 3 se le asigna el trabajo 11, inicialmente la prioridad era 1

El grupo 4 se le asigna el trabajo 5, inicialmente la prioridad era 1

El grupo 5 se le asigna el trabajo 8, inicialmente la prioridad era 2

El grupo 6 se le asigna el trabajo 6, inicialmente la prioridad era 1

El grupo 7 se le asigna el trabajo 7, inicialmente la prioridad era 2

El grupo 8 se le asigna el trabajo 9, inicialmente la prioridad era 2

El grupo 9 se le asigna el trabajo 2, inicialmente la prioridad era 1

El grupo 10 se le asigna el trabajo 13, inicialmente la prioridad era 3

El grupo 11 se le asigna el trabajo 3, inicialmente la prioridad era 4

